In [1]:
from collections import namedtuple
import random
# 乱数のシードは章の番号
random.seed(7)

# 品物（Item）は簡単なクラスなのでnamedtupleで作る。
Item = namedtuple('Item', ('name', 'weight', 'price'))

# 品物の個数
num = 20

# 品物を保持するリスト
item_list = []
max_weight = 5
# 品物の個数numより大きな数字にする
max_price = 50

# 値段の候補リストを作り、シャッフル
price_list = list(range(1, max_price+1))
random.shuffle(price_list)

# ランダムに品物を作ってみる。名前は番号
for i in range(num):
    w = random.randint(1, max_weight)
    item = Item(i, w, price_list.pop())
    item_list.append(item)

In [2]:
item_list

[Item(name=0, weight=3, price=21),
 Item(name=1, weight=1, price=10),
 Item(name=2, weight=5, price=26),
 Item(name=3, weight=1, price=42),
 Item(name=4, weight=5, price=4),
 Item(name=5, weight=1, price=5),
 Item(name=6, weight=5, price=35),
 Item(name=7, weight=2, price=7),
 Item(name=8, weight=4, price=24),
 Item(name=9, weight=5, price=38),
 Item(name=10, weight=4, price=46),
 Item(name=11, weight=3, price=33),
 Item(name=12, weight=4, price=14),
 Item(name=13, weight=5, price=3),
 Item(name=14, weight=4, price=6),
 Item(name=15, weight=3, price=28),
 Item(name=16, weight=3, price=27),
 Item(name=17, weight=2, price=45),
 Item(name=18, weight=2, price=16),
 Item(name=19, weight=2, price=37)]

In [3]:
class Knapsack:
    
    def __init__(self, size):
        # このナップサックが保持できる最大の重さ
        self.size = size
        # 現在の重さ
        self.weight = 0
        # 入っているものの価値の総和
        self.value = 0
        # 保持しているItemの配列
        self.items = []
    
    def append(self, item):
        """このナップサックにItemを追加する。"""
        if not self.has_room_for(item):
            raise ValueError('このアイテムは入れられません。重量オーバーです。')
        self.items.append(item)
        self.weight += item.weight
        self.value += item.price
    
    def has_room_for(self, item):
        """引数にとったアイテムを入れる余裕があるかどうかを真偽値で返す。 """
        return self.size >= self.weight + item.weight
    
    def __str__(self):
        val = '重さ {} kg / 価値　{} 万円'.format(self.weight, self.value)
        return val

In [4]:
def greedy(items, size_limit):
    #　単位重さあたりの値段で品物を並び替える。
    sorted_item_list = sorted(items, key=lambda x: x.price/x.weight, reverse=True)
    my_knapsack = Knapsack(size_limit)
    for v in sorted_item_list:
        # 入る余地があるなら品物を入れる。
        try:
            my_knapsack.append(v)
        except ValueError:
            continue
    return my_knapsack

In [5]:
knap_g = greedy(item_list, 40)
print(knap_g)

重さ 39 kg / 価値　407 万円


In [6]:
knap_g.items

[Item(name=3, weight=1, price=42),
 Item(name=17, weight=2, price=45),
 Item(name=19, weight=2, price=37),
 Item(name=10, weight=4, price=46),
 Item(name=11, weight=3, price=33),
 Item(name=1, weight=1, price=10),
 Item(name=15, weight=3, price=28),
 Item(name=16, weight=3, price=27),
 Item(name=18, weight=2, price=16),
 Item(name=9, weight=5, price=38),
 Item(name=0, weight=3, price=21),
 Item(name=6, weight=5, price=35),
 Item(name=8, weight=4, price=24),
 Item(name=5, weight=1, price=5)]

In [7]:
import itertools

def brute_force(items, size_limit):
    #　答えの候補
    candidate = None
    # 0と1を20個並べるすべてのパターンをこれで作れる
    for pattern in itertools.product((0, 1), repeat=len(items)):
        my_box = []
        for i, val in enumerate(pattern):
            if val: my_box.append(item_list[i])
        w = sum([item.weight for item in my_box])
        # ナップサックの重量制限を守れないならループの次へ
        if w > size_limit: continue
        # 総額を計算しこれまでの最高を上回るなら候補として残す
        value = sum([item.price for item in my_box])
        if candidate is None or value > candidate.value:
            knapsack = Knapsack(size_limit)
            for v in my_box:
                knapsack.append(v)
            candidate = knapsack
    return candidate

In [8]:
knap_bf = brute_force(item_list, 40)
print(knap_bf)

重さ 40 kg / 価値　409 万円


In [9]:
def dp(items, size_limit):
    n = len(items)
    # 価値を記録する表を作成 （行が品物、列が許容サイズ）
    table = [[0]*(size_limit+1) for i in range(n+1)]
    # 価値を更新したかどうかを記録するための表
    flag = [[False]*(size_limit+1) for i in range(n+1)]
    # 表を下に進むループ（考慮に入れる品物）
    for i in range(1, n+1):
        # いれるかどうか考えている品物
        target = items[i-1]
        w = target.weight
        # 表を右に進むループ（重さの上限）
        for j in range(1, size_limit+1):
            # 1行上の最適解
            yellow = table[i-1][j]
            table[i][j] = yellow
            # 今の許容範囲jを越えるなら論外
            if w > j: continue
            # ちょうどtarget分の重さが少ないときの最適解
            pink = table[i-1][j-w]
            # この品物を入れたときの価値
            include_this = target.price + pink
            table[i][j] = max(yellow, include_this)
            flag[i][j] = include_this > yellow
    # 後処理：表を右下から遡って入れた品物を調べる
    i = n
    j = size_limit
    my_knapsack = Knapsack(size_limit)
    while i > 0 and j > 0:
        if flag[i][j]:
            # この価値の更新で追加した品物は、i-1
            my_knapsack.append(items[i-1])
            # 表を左へ戻る
            j -= items[i-1].weight
        i -= 1
    return my_knapsack

In [10]:
knap_dp = dp(item_list, 40)
print(knap_dp)

重さ 40 kg / 価値　409 万円


## 練習問題解答

## 7.1

ループの中でcontinueが実行されると、最も内側のループの次のステップへ進む。break分が実行されると、そのロープを抜ける。ナップサックに順々に品物を詰めていき、入らない品物が初めて見つかったとき、breakが実行されると探索がそこで終わってしまう。ナップサックの余った容量には、別の品物が入る可能性は残っているので、返される結果は貪欲法が正しく実行された時よりも悪いものとなる。

ちなみに、このコードではappendメソッドがナップサックに入らない品物を引数にとると例外を発生することがわかっているので、try-except構文を使っているが、次のように書くこともできる。

In [11]:
def greedy2(items, size_limit):
    sorted_item_list = sorted(items, key=lambda x: x.price/x.weight, reverse=True)
    my_knapsack = Knapsack(size_limit)
    for v in sorted_item_list:
        if my_knapsack.has_room_for(v):
            my_knapsack.append(v)
    return my_knapsack

こちらのコードの方が本文中のコード7.3よりすっきりしていて見やすい。ただ、appendメソッドの中でhas_room_forを呼んでいるので、品物を追加する場合は、同じメソッドを2回呼ぶことになる。has_room_forの処理が重い場合は気をつける必要があるが、いまは気にするほどのことはない。どちらが良い実装なのか、という疑問に対する答えは悩ましいところだ。はっきりとどちらが良いという判断がつかない場合も多い。また、メソッドや関数の動作をドキュメントにして残しておくということも重要だ。いまはクラスの実装と利用を両方1人でしているが、誰かが作ったクラスを利用する場合や、自分が作ったクラスを利用してもらう場合もあるだろう。オープンソースの場合は中身を公開できるが、それでもやはりドキュメントを書くことは重要になる。クラスの各メソッドや関数がどのような動作をするのか、わかりやすく利用者に伝わるようにする必要がある。

## 7.2

元の問題との違いは、荷物を分割できるところにあるので、この問題は分割ナップサック問題と呼ばれることもある。分割はdivisionなので、関数名はdiv_knapsackにしよう。この問題は、元の0-1ナップサック問題と比べて驚くほど簡単に最適解を求めることができる。

In [12]:
def div_knapsack(items, size_limit):
    #　単位重さあたりの価値で品物を並び替える。
    sorted_item_list = sorted(items, key=lambda x: x.price/x.weight, reverse=True)
    my_knapsack = Knapsack(size_limit)
    for v in sorted_item_list:
        # 入る余地があるなら品物を入れる。
        try:
            my_knapsack.append(v)
        except ValueError:
            break
    # vの一部を入るだけ入れる。
    w = my_knapsack.size - my_knapsack.weight
    p = v.price * (w / v.weight)
    virtual_item = Item(-1, w, p)
    my_knapsack.append(virtual_item)
    return my_knapsack

In [13]:
print(div_knapsack(item_list, 40))

重さ 40 kg / 価値　412.4 万円


貪欲法の実装を改造してプログラムを作った。単位重さあたりの価値が高い品物から順に追加していき、品物が入らなくなったところで、ループを抜ける。あらたに、ナップサックに残っている容量にピッタリ収まる仮想的な品物を作り、これを追加した。これが分割ナップサック問題の最適解になる。元の問題の最適解を求めるのは難しいが、問題をすこし変形すると簡単に最適解が求まることはよくある。同じ品物リストであれば、分割ナップサック問題の解は、元の0-1ナップサック問題の解よりも価値が高い。この分割ナップサック問題を利用して、本章で扱った総当たりで計算するアルゴリズムの計算量を削減する方法もある。これは一般的には、分岐限定法と呼ばれる方法の1つになる。

## 7.3

やさしいナップサック問題は、その名の通り簡単に解くことができる。0-1ナップサック問題の近似解法として紹介した貪欲法で最適解を求めることができる。問題文の式（7.1）が成り立つので、ある品物$i$がナップサックに入るにも関わらず入れないということは、残りの品物すべてを入れても品物$i$より軽くなる（価値が低くなる）ことを意味する。つまり、貪欲法で最適解を求めることができる。

まずはサンプルデータを用意しよう。

In [14]:
random.seed(7)

#品物の数
num = 10

# 品物を生成するための数を保持するリスト
val_list = []

# 最初の品物の重さ。重さの10％の標準偏差で正規分布に従う乱数を生成
w = random.gauss(2, 2*0.1)
val_list.append(w)

for i in range(num):
    total_w = sum(val_list)
    w = total_w + abs(random.gauss(total_w * 0.1, 1))
    val_list.append(w)

tender_list = []
for i, v in enumerate(val_list):
    tender_list.append(Item(i, v, v))

tender_list

[Item(name=0, weight=1.9488239423104798, price=1.9488239423104798),
 Item(name=1, weight=2.6551378490580415, price=2.6551378490580415),
 Item(name=2, weight=4.838261805722269, price=4.838261805722269),
 Item(name=3, weight=10.071377534468683, price=10.071377534468683),
 Item(name=4, weight=20.534943054392652, price=20.534943054392652),
 Item(name=5, weight=43.84009665712613, price=43.84009665712613),
 Item(name=6, weight=93.38942230837242, price=93.38942230837242),
 Item(name=7, weight=195.43001615072168, price=195.43001615072168),
 Item(name=8, weight=411.01576631127926, price=411.01576631127926),
 Item(name=9, weight=862.345132902462, price=862.345132902462),
 Item(name=10, weight=1811.0706460021188, price=1811.0706460021188)]

重さ（値段）に関する条件は、かなり急速に数値が多きくなることがわかる。いくつかの品物が入るように、ナップサックのサイズを3,000としよう。貪欲法のアルゴリズムを少し改造する。

In [15]:
def greedy_for_tender(items, size_limit):
    #　重さ（または値段）で品物を並び替える。
    sorted_item_list = sorted(items, key=lambda x: x.weight, reverse=True)
    my_knapsack = Knapsack(size_limit)
    for v in sorted_item_list:
        # 入る余地があるなら品物を入れる。
        try:
            my_knapsack.append(v)
        except ValueError:
            continue
    return my_knapsack

In [16]:
answer_knapsack = greedy_for_tender(tender_list, 3000)

In [17]:
print(answer_knapsack)

重さ 2999.628623700569 kg / 価値　2999.628623700569 万円


In [18]:
answer_knapsack.items

[Item(name=10, weight=1811.0706460021188, price=1811.0706460021188),
 Item(name=9, weight=862.345132902462, price=862.345132902462),
 Item(name=7, weight=195.43001615072168, price=195.43001615072168),
 Item(name=6, weight=93.38942230837242, price=93.38942230837242),
 Item(name=4, weight=20.534943054392652, price=20.534943054392652),
 Item(name=3, weight=10.071377534468683, price=10.071377534468683),
 Item(name=2, weight=4.838261805722269, price=4.838261805722269),
 Item(name=0, weight=1.9488239423104798, price=1.9488239423104798)]